In [1]:
import numpy as np
import keras
from skimage.measure import block_reduce 
from keras.models import load_model,Model,Sequential
from keras.layers import Dense,Dropout,Flatten,Activation
from keras import regularizers
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
if K.backend()=='tensorflow':
    K.set_image_dim_ordering("tf")
    
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

Using TensorFlow backend.


In [8]:
############################################
#Feature Extraction using numpy pooling
############################################
#which base model to extract features
#which layer(name) to extract features
#X: input data(a generator)
#steps: steps for predict_generator 
#size: block_size for image downsampling
def extract_features(base, name, X,step,size):
    target = Model(inputs=base.input,outputs=base.get_layer(name).output)
    features = target.predict_generator(X,steps=steps,verbose=1)
    n,a,b,c=features.shape
    features1=[]
    for i in range(n):
        new=block_reduce(features[i,:,:,:], block_size=(size,size,1), func=np.mean)
        a,b,c=new.shape
        features1.append(new.reshape(a*b*c))
    features1=np.stack(features1)
    return features1

In [9]:
val_generator = test_datagen.flow_from_directory(
    '/pylon5/ms3uujp/dx10384/caltech101/test',
#    '/pylon5/ms3uujp/dx10384/caltech101/101_ObjectCategories',
    target_size = (299, 299),
    color_mode = 'rgb',
    batch_size = 1,
    shuffle=False)

Found 1359 images belonging to 102 classes.


In [10]:
label= val_generator.classes
steps=len(label)

In [11]:
base = InceptionV3(weights='imagenet', include_top=False, input_shape = (299,299,3))

In [12]:
feature1=extract_features(base,'mixed0',val_generator,steps,35)
feature2=extract_features(base,'mixed1',val_generator,steps,35)
feature3=extract_features(base,'mixed2',val_generator,steps,35)
feature4=extract_features(base,'mixed3',val_generator,steps,17)
feature5=extract_features(base,'mixed4',val_generator,steps,17)
feature6=extract_features(base,'mixed5',val_generator,steps,17)
feature7=extract_features(base,'mixed6',val_generator,steps,17)
feature8=extract_features(base,'mixed7',val_generator,steps,17)
feature9=extract_features(base,'mixed8',val_generator,steps,8)
feature10=extract_features(base,'mixed9',val_generator,steps,8)
feature11=extract_features(base,'mixed10',val_generator,steps,8)

1359/1359 [==============================] - 28s 21ms/step


In [13]:
##################################################
#Projection correlation 
##################################################

def get_arccos_1d(X):

    # X -- a 1D array
    
    X = np.squeeze(X)
    Y = X[:,None] - X
    Z = Y.T[:,:,None]*Y.T[:,None]
    n = len(X)
    
    a = np.zeros([n, n, n])
    a[Z == 0.] = np.pi/2.
    a[Z < 0.] = np.pi
    
    a = np.transpose(a, (1,2,0))

    a_bar_12 = np.mean(a, axis = 0, keepdims = True)
    a_bar_02 = np.mean(a, axis = 1, keepdims = True)
    a_bar_2  = np.mean(a, axis = (0,1), keepdims = True)
    A = a - a_bar_12 - a_bar_02 + a_bar_2
    
    return a, A


def get_arccos(X):

    # X -- a 2D array
    
    n, p = X.shape
    cos_a = np.zeros([n, n, n])
    
    for r in range(n):
        
        xr = X[r]
        X_r = X - xr
        cross = np.dot(X_r, X_r.T)
        row_norm = np.sqrt(np.sum(X_r**2, axis = 1))
        outer_norm = np.outer(row_norm, row_norm)
        
        zero_idx = (outer_norm == 0.)
        outer_norm[zero_idx] = 1.
        cos_a_kl = cross / outer_norm
        cos_a_kl[zero_idx] = 0.

        cos_a[:,:,r] = cos_a_kl
        
    cos_a[cos_a > 1] = 1.
    cos_a[cos_a < -1] = -1.
    a = np.arccos(cos_a)

    a_bar_12 = np.mean(a, axis = 0, keepdims = True)
    a_bar_02 = np.mean(a, axis = 1, keepdims = True)
    a_bar_2  = np.mean(a, axis = (0,1), keepdims = True)
    A = a - a_bar_12 - a_bar_02 + a_bar_2
        
    return a, A

def projection_corr_1dy(X, Y):

    """
    compute the projection correlation where
    X -- an n*p 2D array
    Y -- an n*1 2D array
    """
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos(X)
    a_y, A_y = get_arccos_1d(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

In [15]:
j=1
from sklearn.utils import shuffle
nsample=np.array([500,1000,1500])
subfeature1,sublabel1=shuffle(feature1,label,n_samples=nsample[j])
subfeature2,sublabel2=shuffle(feature2,label,n_samples=nsample[j])
subfeature3,sublabel3=shuffle(feature3,label,n_samples=nsample[j])
subfeature4,sublabel4=shuffle(feature4,label,n_samples=nsample[j])
subfeature5,sublabel5=shuffle(feature5,label,n_samples=nsample[j])
subfeature6,sublabel6=shuffle(feature6,label,n_samples=nsample[j])
subfeature7,sublabel7=shuffle(feature7,label,n_samples=nsample[j])
subfeature8,sublabel8=shuffle(feature8,label,n_samples=nsample[j])
subfeature9,sublabel9=shuffle(feature9,label,n_samples=nsample[j])
subfeature10,sublabel10=shuffle(feature10,label,n_samples=nsample[j])
subfeature11,sublabel11=shuffle(feature11,label,n_samples=nsample[j])

In [ ]:
print (projection_corr_1dy(subfeature1,sublabel1.reshape(-1,1)))
print (projection_corr_1dy(subfeature2,sublabel2.reshape(-1,1)))
print (projection_corr_1dy(subfeature3,sublabel3.reshape(-1,1)))
print (projection_corr_1dy(subfeature4,sublabel4.reshape(-1,1)))
print (projection_corr_1dy(subfeature5,sublabel5.reshape(-1,1)))
print (projection_corr_1dy(subfeature6,sublabel6.reshape(-1,1)))
print (projection_corr_1dy(subfeature7,sublabel7.reshape(-1,1)))
print (projection_corr_1dy(subfeature8,sublabel8.reshape(-1,1)))
print (projection_corr_1dy(subfeature9,sublabel9.reshape(-1,1)))
print (projection_corr_1dy(subfeature10,sublabel10.reshape(-1,1)))
print (projection_corr_1dy(subfeature11,sublabel11.reshape(-1,1)))

0.18678991155780364
0.19121241992449575
0.23617342279109474
0.2580383245790337
0.3244303306536486
0.33725607228313753
0.37315819275569667
0.3251593439002487
